High Value Customer Identification (Insiders)

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [16]:
data = pd.read_csv("df_products.csv")
data.head(2)

,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,size,style_id,color_id
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-30 05:56:05,NaN,1024256001,Only non-chlorine bleach when needed,NaN,"Shell: Cotton 99%, Spandex 1%",DENIM,Black,Slim fit,Yes,Denim,NaN,1024256,1
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-11-30 05:56:05,NaN,1024256001,Medium iron,NaN,"Pocket lining: Polyester 65%, Cotton 35%",DENIM,Solid-color,Slim fit,Yes,Denim,NaN,1024256,1


In [17]:
data2 = data.copy()

for i in ['style_id', 'color_id', 'product_id']:
    data2[i] = data2[i].astype(int)

for i in ['product_name', 'fit', 'description', 'material']:
    data2[i] = data2[i].apply( lambda x: x.replace(" ", "_").lower())
    data2[i] = data2[i].apply( lambda x: x.replace("/", "_").lower())

data2['composition'] = data2['composition'].str.lower()

data2['product_price'] = data2['product_price'].apply( lambda x: x.replace("$", "")).astype(float)

data2['scrapy_datetime'] = pd.to_datetime(data2['scrapy_datetime'], format = "%Y-%m-%d %H:%M:%S")

data2['size_number'] = data2['size'].apply( lambda x: re.search("\d+[^cm]*", x).group(0) if pd.notnull( x ) else x )

data2['size_model'] = data2['size'].str.extract( '(\d+\/\d+)' )

data2 = data2.drop(['size'], axis=1)


print(data2.shape)

for i in ['lining', 'shell', 'pocket']:
    data2 = data2[~data2['composition'].str.contains( i, na=False) ]

data2 = data2.drop_duplicates( subset=)

data2 = data2.reset_index()

print(data2.shape)


data2.head(2)

(68, 19)
(8, 19)


,product_id,product_category,product_name,product_price,scrapy_datetime,additional material information,art. no.,care instructions,collection,composition,concept,description,fit,imported,material,style_id,color_id,size_number,size_model
8,690449022,men_jeans_ripped,skinny_jeans,39.99,2022-11-30 05:56:05,NaN,690449022,Only non-chlorine bleach when needed,NaN,"cotton 98%, spandex 2%",DENIM,black,skinny_fit,Yes,keyfibretype,690449,22,NaN,NaN
20,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-11-30 05:56:05,NaN,690449043,Only non-chlorine bleach when needed,NaN,"cotton 98%, spandex 2%",DENIM,light_denim_blue,skinny_fit,Yes,keyfibretype,690449,43,NaN,NaN
